In [27]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [28]:
# Carregar os arquivos CSV combinados
arq_combined = pd.read_csv('../model/data/modisTodos.csv')
arq_combined.head()

,latitude,longitude,confidence
0,-33.7092,-53.3442,0.0
1,-33.6991,-53.3287,0.0
2,-33.3994,-52.9535,0.0
3,-33.3934,-52.9476,0.0
4,-33.3688,-52.9170,0.0


In [29]:
# Criar colunas futuras (latitude_futuro, longitude_futuro, confidence_futuro)
arq_combined['latitude_futuro'] = arq_combined['latitude'].shift(-1)
arq_combined['longitude_futuro'] = arq_combined['longitude'].shift(-1)
arq_combined['confidence_futuro'] = arq_combined['confidence'].shift(-1)

In [30]:
# Remover linhas com valores NaN
arq_combined.dropna(inplace=True)

In [31]:
# Separar as colunas de entrada (features) e saída (targets)
X = arq_combined[['latitude', 'longitude', 'confidence']].values
y = arq_combined[['latitude_futuro', 'longitude_futuro', 'confidence_futuro']].values

In [32]:
# Dividir em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [33]:
# Normalizar os dados
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [34]:
# Definir a arquitetura da rede neural
model = tf.keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(128, activation='relu'),
    layers.Dense(3)
])

/Users/u22124/Library/Python/3.9/lib/python/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [35]:
# Compilar o modelo
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

In [36]:
# Treinar o modelo
history = model.fit(X_train, y_train, epochs=50, validation_split=0.2)

Epoch 1/50
20975/20975 ━━━━━━━━━━━━━━━━━━━━ 10s 460us/step - loss: 202.1483 - mae: 7.1317 - val_loss: 135.2585 - val_mae: 5.7217
Epoch 2/50
20975/20975 ━━━━━━━━━━━━━━━━━━━━ 9s 450us/step - loss: 134.1036 - mae: 5.6539 - val_loss: 133.1966 - val_mae: 5.6148
Epoch 3/50
20975/20975 ━━━━━━━━━━━━━━━━━━━━ 10s 457us/step - loss: 132.7583 - mae: 5.6160 - val_loss: 134.0683 - val_mae: 5.6910
Epoch 4/50
20975/20975 ━━━━━━━━━━━━━━━━━━━━ 10s 452us/step - loss: 133.1159 - mae: 5.6220 - val_loss: 132.7931 - val_mae: 5.6801
Epoch 5/50
20975/20975 ━━━━━━━━━━━━━━━━━━━━ 10s 453us/step - loss: 132.4817 - mae: 5.6005 - val_loss: 133.6315 - val_mae: 5.6308
Epoch 6/50
20975/20975 ━━━━━━━━━━━━━━━━━━━━ 9s 447us/step - loss: 132.8069 - mae: 5.5960 - val_loss: 133.3076 - val_mae: 5.5445
Epoch 7/50
20975/20975 ━━━━━━━━━━━━━━━━━━━━ 10s 454us/step - loss: 132.4419 - mae: 5.5850 - val_loss: 132.4704 - val_mae: 5.6773
Epoch 8/50
20975/20975 ━━━━━━━━━━━━━━━━━━━━ 10s 454us/step - loss: 132.3260 - mae: 5.5739 - val_los

In [37]:
# Avaliar o modelo no conjunto de teste
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f'Mean Absolute Error on Test Data: {test_mae}')

6555/6555 ━━━━━━━━━━━━━━━━━━━━ 2s 272us/step - loss: 132.0351 - mae: 5.5404
Mean Absolute Error on Test Data: 5.532376766204834


In [38]:
# Fazer previsões com novos dados
y_pred = model.predict(X_test)
final = y_pred[:100] # Exibir as primeiras 5 previsões

6555/6555 ━━━━━━━━━━━━━━━━━━━━ 2s 252us/step


In [39]:
# Criar um DataFrame do pandas com os dados
final = pd.DataFrame(final, columns=['latitude_futuro', 'longitude_futuro', 'confidence_futuro'])

# Salvar o DataFrame em um arquivo CSV
final.to_csv('valores_futuros.csv', index=False)